In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity


In [2]:
anime = pd.read_csv("anime.csv")

In [3]:
ratings = pd.read_csv("animelist.csv",nrows=50000000)

In [4]:
anime = anime.loc[anime['Type'] == 'TV']

In [16]:
anime.head()

,MAL_ID,Name,Score,Genres,English name,Japanese name,Type,Episodes,Aired,Premiered,...,Score-10,Score-9,Score-8,Score-7,Score-6,Score-5,Score-4,Score-3,Score-2,Score-1
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",Cowboy Bebop,カウボーイビバップ,TV,26,"Apr 3, 1998 to Apr 24, 1999",Spring 1998,...,229170.0,182126.0,131625.0,62330.0,20688.0,8904.0,3184.0,1357.0,741.0,1580.0
2,6,Trigun,8.24,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen",Trigun,トライガン,TV,26,"Apr 1, 1998 to Sep 30, 1998",Spring 1998,...,50229.0,75651.0,86142.0,49432.0,15376.0,5838.0,1965.0,664.0,316.0,533.0
3,7,Witch Hunter Robin,7.27,"Action, Mystery, Police, Supernatural, Drama, ...",Witch Hunter Robin,Witch Hunter ROBIN (ウイッチハンターロビン),TV,26,"Jul 2, 2002 to Dec 24, 2002",Summer 2002,...,2182.0,4806.0,10128.0,11618.0,5709.0,2920.0,1083.0,353.0,164.0,131.0
4,8,Bouken Ou Beet,6.98,"Adventure, Fantasy, Shounen, Supernatural",Beet the Vandel Buster,冒険王ビィト,TV,52,"Sep 30, 2004 to Sep 29, 2005",Fall 2004,...,312.0,529.0,1242.0,1713.0,1068.0,634.0,265.0,83.0,50.0,27.0
5,15,Eyeshield 21,7.95,"Action, Sports, Comedy, Shounen",Unknown,アイシールド21,TV,145,"Apr 6, 2005 to Mar 19, 2008",Spring 2005,...,9226.0,14904.0,22811.0,16734.0,6206.0,2621.0,795.0,336.0,140.0,151.0


In [6]:
ratings = ratings.loc[ratings['anime_id'].isin(anime['MAL_ID'])]
ratings.shape

(34476048, 5)

In [7]:
ratings = ratings.groupby('user_id').filter(lambda x : len(x)>10)
print(ratings.shape)

(34438062, 5)


In [8]:
def GetAnime(ID):
    row = anime.loc[anime['MAL_ID'] == ID]
    return row['Name'].to_string(index=False)

print(GetAnime(7))

Witch Hunter Robin


In [9]:
ratings.drop(['watching_status', 'watched_episodes'], axis=1, inplace=True)

In [10]:
users = ratings['user_id'].unique()

UserMeans = ratings.groupby(['user_id']).mean()
UserMeans

,anime_id,rating
user_id,,
0,1568.372549,4.352941
1,26127.711712,6.747748
2,19239.023392,2.345029
3,16279.968750,6.762500
4,2253.699187,6.032520
...,...,...
162164,15684.984436,6.136187
162165,13935.808511,5.840426
162166,22964.093458,3.929907


In [11]:
UserMeans.loc[0]

anime_id    1568.372549
rating         4.352941
Name: 0, dtype: float64

In [27]:
def GetRelevant(ID):
 
    RatingsOfAnime = ratings.loc[ratings['anime_id'] == ID]
    RelevantUsers = RatingsOfAnime['user_id']
    
    
    Relevant = ratings.loc[ratings['user_id'].isin(RelevantUsers)]
    print(Relevant)
    return Relevant


In [31]:
GetRelevant(6)

          user_id  anime_id  rating
1416            6      9919       0
1417            6       150       7
1418            6      4981       0
1419            6     10793       0
1420            6       249       7
...           ...       ...     ...
49998857   162163      3712       7
49998858   162163     11837       6
49998859   162163     17681       8
49998860   162163        29       7
49998862   162163       855       5

[14778683 rows x 3 columns]


,user_id,anime_id,rating
1416,6,9919,0
1417,6,150,7
1418,6,4981,0
1419,6,10793,0
1420,6,249,7
...,...,...,...
49998857,162163,3712,7
49998858,162163,11837,6
49998859,162163,17681,8
49998860,162163,29,7


In [13]:
def BuildRecs():
    
    Relevant = ratings[0:1000000] 
    Relevant = Relevant.groupby('anime_id').filter(lambda x : len(x)>100)
    Relevant['rating'] = Relevant.apply(lambda x: x['rating']-UserMeans.loc[x['user_id']]['rating'], axis = 1)
    Matrix = pd.crosstab(Relevant.user_id , Relevant.anime_id, Relevant.rating, aggfunc='sum')  
    Matrix = Matrix.fillna(0)  
    SimilarityNP = cosine_similarity(Matrix.T)  
    Similarity = pd.DataFrame(SimilarityNP,index = Matrix.columns,columns = Matrix.columns)
     
    return Similarity


In [32]:
Similarity = BuildRecs()
print(Similarity)

anime_id     1         6         7         15        16        18     \
anime_id                                                               
1         1.000000  0.167881 -0.053264 -0.035309  0.005341  0.049665   
6         0.167881  1.000000  0.003759 -0.053335  0.009211  0.033195   
7        -0.053264  0.003759  1.000000  0.024086  0.031142 -0.058225   
15       -0.035309 -0.053335  0.024086  1.000000  0.018455  0.040777   
16        0.005341  0.009211  0.031142  0.018455  1.000000 -0.010707   
...            ...       ...       ...       ...       ...       ...   
42938    -0.009252 -0.015888  0.021539  0.008878  0.055359 -0.027165   
42963     0.011582  0.013684  0.023215  0.004626  0.024210 -0.027604   
43299    -0.020565  0.008322  0.019087  0.019615  0.027077 -0.003704   
43608     0.009510  0.006452  0.012441 -0.009001  0.022642 -0.031837   
47778    -0.021405  0.015481  0.022904  0.010057  0.022185 -0.046614   

anime_id     19        20        21        22     ...     42205

In [57]:

NewAnime = anime.filter(['MAL_ID','Name'], axis=1)
NewAnime.rename(columns = {'MAL_ID' : 'anime_id'}, inplace = True)
    
# print(NewAnime[NewAnime['Name'] == 'Naruto'].index[0])
NewAnime.head()

,anime_id,Name
0,1,Cowboy Bebop
2,6,Trigun
3,7,Witch Hunter Robin
4,8,Bouken Ou Beet
5,15,Eyeshield 21


In [111]:
def RecAnime(Name,Rating):
    
    
    SimilarScores = Similarity[Name]*10
    SimilarScores = SimilarScores.sort_values(ascending = False)
    SimilarScoresDF = pd.DataFrame(SimilarScores)
    SimilarScoresDF.columns = ['Similarity Score']
    
   
    
    SimilarScoresDF = SimilarScoresDF[1:11]

    SimilarScoresDF = pd.merge(SimilarScoresDF, NewAnime, on="anime_id", how="inner")
    
    print(SimilarScoresDF['Name'])

 


  
    


In [113]:
def Aname(GetName):
    score = NewAnime[NewAnime['Name'] == GetName].index[0]
    RecAnime(score,8)


10


KeyError: 2

In [ ]:
import pickle

In [ ]:
pickle.dump(Similarity,open('Similarity.pkl','wb'))